# Vulnerability analysis of popular CNN models to data poisoning attacks (single pixel, pixel pattern, image injection)

**Install ART framwerok**

In [1]:
!pip install adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.2 MB/s eta 0:00:00


**Install dependecies**

In [2]:
import numpy as np
import torch
from art.estimators.classification import PyTorchClassifier
from art.attacks.poisoning import PoisoningAttackBackdoor
from art.attacks.poisoning.perturbations import add_single_bd, add_pattern_bd, insert_image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from art.utils import load_dataset

**Define poisoning attack types**

In [3]:
def poison_single_pixel(x):
    return add_single_bd(x, distance=12,)

def poison_pattern_pixel(x):
    return add_pattern_bd(x, distance=2, channels_first=True)

def poison_insert(x):
    return insert_image(
        x,
        backdoor_path='trigger.png',
        channels_first=True,
        random=False,
        x_shift=0,
        y_shift=0,
        size=(8, 8),
        mode='RGB',
        blend=0.8
    )



**Import data set and pretrained models**

In [4]:
test_models = {
    'Resnet56': lambda: torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet56", pretrained=True),
    'VGG19': lambda: torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg19_bn", pretrained=True),
    'MobileNetV2': lambda: torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_mobilenetv2_x1_4", pretrained=True),
    'repvgg_a2': lambda: torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_repvgg_a2", pretrained=True),
}



(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset('cifar10')

x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.201)

**Poison data and evaluate models**

In [5]:


def poisoning_samples(x_to_poison, y_to_poison, poison_percent, attack_type, source_class, target_class):

  x_poison = np.copy(x_to_poison)
  y_poison = np.copy(y_to_poison)
  is_poison = np.zeros(len(x_to_poison)).astype(bool)

  indices = np.where(y_to_poison == source_class)[0]
  num_poison = int(poison_percent * len(indices))

  for i in indices[:num_poison]:
      x_poison[i], _ = attack_type.poison(x_poison[i], [])
      y_poison[i] = target_class
      is_poison[i] = True

  poison_indices = np.where(is_poison)[0]
  print('Ilość zatrutych przykładów:', np.sum(is_poison))
  return x_poison, y_poison, poison_indices


def evaluate_metrics(classifier, x_test, y_test):
  predictions = classifier.predict(x_test)
  predicted_classes = np.argmax(predictions, axis=1)
  accuracy = accuracy_score(y_test, predicted_classes)
  precision = precision_score(y_test, predicted_classes, average='macro')
  recall = recall_score(y_test, predicted_classes, average='macro')
  f1 = f1_score(y_test, predicted_classes, average='macro')

  print(f"  \nMiary dla {model_name}:")
  print(f"  Dokładność: {accuracy:.4f}")
  print(f"  Precyzja: {precision:.4f}")
  print(f"  Czułość: {recall:.4f}")
  print(f"  F1: {f1:.4f}")
  print(f"\n")



attacks_bd = [
    PoisoningAttackBackdoor(poison_single_pixel),
    PoisoningAttackBackdoor(poison_pattern_pixel),
    PoisoningAttackBackdoor(poison_insert),
]


source_class = 6
target_class = 4
poison_percent = 0.1

labels = ['samolot', 'samochód', 'ptak', 'kot', 'jeleń', 'pies', 'żaba', 'koń', 'statek', 'ciężarówka']



for model_name,  model_fn in test_models.items():
  for attack in attacks_bd:
    print(f"\nTrenowanie modelu: {model_name}")
    model = model_fn()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)
    loss_fn = torch.nn.CrossEntropyLoss()
    classifier = PyTorchClassifier(
        model=model,
        loss=loss_fn,
        optimizer=optimizer,
        input_shape=(3, 32, 32),
        nb_classes=10,
        clip_values=(min_, max_),
        preprocessing = (mean, std)
    )
    evaluate_metrics(classifier, x_test, y_test)

    x_poisoned, y_poisoned, poisoned_indices = poisoning_samples(x_train, y_train, poison_percent, attack, source_class, target_class)
    classifier.fit(x_poisoned, y_poisoned, nb_epochs=2)
    print("Miary po zatruciu:")
    evaluate_metrics(classifier, x_test, y_test)
    print("Skuteczność ataku zatrucia:")
    x_test_poisoned, y_test_poisoned, test_poisoned_indices = poisoning_samples(x_test, y_test, 1, attack, source_class, target_class)
    predictions = classifier.predict(x_test_poisoned[test_poisoned_indices])
    predicted_classes = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(np.full_like(predicted_classes, fill_value=target_class), predicted_classes)
    print(f"  Dokładność : {accuracy:.4f}")



Trenowanie modelu: Resnet56


/usr/local/lib/python3.12/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/releases/download/resnet/cifar10_resnet56-187c023a.pt" to /root/.cache/torch/hub/checkpoints/cifar10_resnet56-187c023a.pt


100%|██████████| 3.39M/3.39M [00:00<00:00, 102MB/s]


  
Miary dla Resnet56:
  Dokładność: 0.9437
  Precyzja: 0.9436
  Czułość: 0.9437
  F1: 0.9436


Ilość zatrutych przykładów: 500
Miary po zatruciu:
  
Miary dla Resnet56:
  Dokładność: 0.8731
  Precyzja: 0.8791
  Czułość: 0.8731
  F1: 0.8730


Skuteczność ataku zatrucia:
Ilość zatrutych przykładów: 1000
  Dokładność : 0.9220

Trenowanie modelu: Resnet56


...

  
Miary dla repvgg_a2:
  Dokładność: 0.9527
  Precyzja: 0.9527
  Czułość: 0.9527
  F1: 0.9526


Ilość zatrutych przykładów: 500
Miary po zatruciu:
  
Miary dla repvgg_a2:
  Dokładność: 0.8073
  Precyzja: 0.8351
  Czułość: 0.8073
  F1: 0.8094


Skuteczność ataku zatrucia:
Ilość zatrutych przykładów: 1000
  Dokładność : 0.9600

Trenowanie modelu: repvgg_a2


Using cache found in /root/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


  
Miary dla repvgg_a2:
  Dokładność: 0.9527
  Precyzja: 0.9527
  Czułość: 0.9527
  F1: 0.9526


Ilość zatrutych przykładów: 500
Miary po zatruciu:
  
Miary dla repvgg_a2:
  Dokładność: 0.8165
  Precyzja: 0.8370
  Czułość: 0.8165
  F1: 0.8153


Skuteczność ataku zatrucia:
Ilość zatrutych przykładów: 1000
  Dokładność : 0.9560

Trenowanie modelu: repvgg_a2


Using cache found in /root/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


  
Miary dla repvgg_a2:
  Dokładność: 0.9527
  Precyzja: 0.9527
  Czułość: 0.9527
  F1: 0.9526


Ilość zatrutych przykładów: 500
Miary po zatruciu:
  
Miary dla repvgg_a2:
  Dokładność: 0.8547
  Precyzja: 0.8606
  Czułość: 0.8547
  F1: 0.8544


Skuteczność ataku zatrucia:
Ilość zatrutych przykładów: 1000
  Dokładność : 1.0000
